In [1]:
import os
import json
import spacy
from tqdm.notebook import tqdm
from spacy.scorer import Scorer
from spacy.training import Example

In [2]:
gpu = spacy.prefer_gpu()
print(gpu)

True


In [3]:
nlp = spacy.load('../training-output/v2/model-best')

In [4]:
with open('../label-studio-files/v1/output.json') as f:
    labeled_data = json.load(f)

In [5]:
training_labeled_data = []
for data in labeled_data:
    text = data["data"]["text"]
    entities = []
    for annotation in data["annotations"][0]["result"]:
        start = annotation["value"]["start"]
        end = annotation["value"]["end"]
        label = annotation["value"]["labels"][0]
        for ent in entities:
            # check if the current entity is subspan of the annotated entity
            if (start >= ent[0] and end <= ent[1]) or (start <= ent[0] and end >= ent[1]):
                break
        else:
            entities.append((start, end, label))
    training_labeled_data.append((text, {"entities": entities}))

In [6]:
scorer = Scorer(nlp)

In [7]:
examples = []
for text, entities in training_labeled_data[:1]:
    predicted = nlp(text)
    example = Example.from_dict(predicted, entities)
    examples.append(example)
scorer.score(examples)

{'token_acc': 1.0,
 'token_p': 1.0,
 'token_r': 1.0,
 'token_f': 1.0,
 'ents_p': 0.6666666666666666,
 'ents_r': 0.6,
 'ents_f': 0.631578947368421,
 'ents_per_type': {'ROLE': {'p': 1.0, 'r': 1.0, 'f': 1.0},
  'SOFT_SKILL': {'p': 1.0, 'r': 0.75, 'f': 0.8571428571428571},
  'TECH_SKILL': {'p': 0.53125,
   'r': 0.7727272727272727,
   'f': 0.6296296296296297},
  'DEGREE': {'p': 1.0, 'r': 0.6666666666666666, 'f': 0.8},
  'PROG_LANG': {'p': 1.0, 'r': 1.0, 'f': 1.0},
  'RESP': {'p': 0.0, 'r': 0.0, 'f': 0.0},
  'REQ': {'p': 0.0, 'r': 0.0, 'f': 0.0},
  'OTHER': {'p': 0.0, 'r': 0.0, 'f': 0.0}}}